# Reinforcement Learning

# Question 1: Describe the environment in the Nim learning model.

The environment in the Nim learning model consists of a given number of piles.  There is a maximum number of items each pile can contain.  This example has three piles and each of these can exist in 11 different states (0 to 10 items in each).  The piles and their contents make up the environment.

# Question 2: Describe the agent(s) in the Nim learning model.

The agents in the Nim learning model are the two players in each game.  This could be any combination of the random player, the Guru, and the Q-Learner.

# Question 3: Describe the reward and penalty in the Nim learning model.

The Nim learning model could use reward and penalty at a few different places.  First, the learning model could received a reward when it wins a game and a penalty when it loses a game.  Next, the individual steps that led to a win or a loss could receive a reward or penalty.  For example, when a player removes the last items from all of the piles, a reward would be given.  If the next move led to a win, a penalty would be given since this means the original player lost.

# Question 4: How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?

There can be 0 to 10 items in each of the 3 piles at any given time.  This means there are 11^3 possible states.  11^3 = 1331.

**Possible States = 1331**

# Question 5: How many possible unique actions there could be for player 1 in the Nim game with 10 items per pile and 3 piles total?

For their first turn, player 1 could take anywhere from 1 to 10 items out of any of the 3 piles.  So there are 30 different actions they could take.  Subsequent possible actions would depend on the move of player 2.

# Question 6: Find a way to improve the provided Nim game learning model. Do you think one can beat the Guru player?

In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration

    return move, pile  # action

In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side

        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")

    return wins['A'], wins['B']

In [4]:
qtable, Alpha, Gamma, Reward, Loss = None, 1.0, 0.8, 100.0, -100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished              
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game
       
            else:
                # find next best move based on state and qtable
                move, pile = nim_qlearner(st2)    
                st3 = st2
                
                # make the next move
                st3[pile] -= move
                
                if st3 == [0, 0, 0]:
                    qtable_update(Loss, st2, move, pile, 0)  # I lost
                    break  # new game
                    
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [6]:
# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]

1000 games, Qlearner  490    Random  510
1000 games, Qlearner  558    Random  442
1000 games, Qlearner  724    Random  276
1000 games, Qlearner  823    Random  177
1000 games, Qlearner  811    Random  189
1000 games, Qlearner  828    Random  172
1000 games, Qlearner  807    Random  193


**From previous program:**

Qlearner 508       Random 492

Qlearner 561       Random 439

Qlearner 713       Random 287

Qlearner 728       Random 272

Qlearner 717       Random 283

Qlearner 719       Random 281

Qlearner 731       Random 269

These adjustments improve the Qlearner when training sizes are at 1000 games.  In this case, the Qlearner wins about 100 games more than the previous method.  This program checks an additional step than the original.  If a move does not result in a win, the next move is checked for a win.  If this move results in a win, the Qlearner gets a loss.  A penalty of -100 is then assigned to the Qtable for this move.